## Google Colab Rust Setup

The following cell is used to set up a Rust environment on Colab. Don't execute it locally!

Many thanks to [`mateusvmv`](https://github.com/mateusvmv) for this hack in [`gist.github.com/korakot/ae95315ea6a3a3b33ee26203998a59a3`](https://gist.github.com/korakot/ae95315ea6a3a3b33ee26203998a59a3?permalink_comment_id=4715636#gistcomment-4715636).

In [ ]:
# This script sets up and spins up a Jupyter Notebook environment with a Rust kernel using Nix and IPC Proxy. 
!wget -qO- https://gist.github.com/wiseaidev/2af6bef753d48565d11bcd478728c979/archive/3f6df40db09f3517ade41997b541b81f0976c12e.tar.gz | tar xvz --strip-components=1
!bash setup_evcxr_kernel.sh

## The Derivative Function

#### $y = x^3 - 2x^2 + 3x - 4$

In [2]:
fn f(x: f64) -> f64 {
    x.powi(3) - 2.0 * x.powi(2) + 3.0 * x - 4.0
}

In [3]:
fn derivative(f: fn(f64) -> f64, x: f64) -> f64 {
    let delta_x = 1.0 / 1_000_000.0;
    (f(x + delta_x) - f(x)) / delta_x
}

In [4]:
let x_value = 2.0;
let result = derivative(f, x_value);
println!("The derivative at x = {} is: {}", x_value, result);

The derivative at x = 2 is: 7.000004001334048


### Example 1: Exploring a Trigonometric Function

#### $y = \sin(x)$

In [5]:
fn sin_function(x: f64) -> f64 {
    x.sin()
}

println!("Derivatives of sin(x):");
for i in -3..=3 {
    println!("{} {}", i, derivative(sin_function, i as f64));
}

Derivatives of sin(x):
-3 -0.989992426175812
-2 -0.4161463817986544
-1 0.540302726670383
0 0.9999999999998334
1 0.5403018851213304
2 -0.41614729129335615
3 -0.9899925672851584


()

### Example 2: Analyzing an Exponential Growth Function

In [6]:
fn exponential_function(x: f64) -> f64 {
    x.exp()
}

println!("Derivatives of e^x:");
for i in -3..=3 {
    println!("{} {}", i, derivative(exponential_function, i as f64));
}

Derivatives of e^x:
-3 0.04978709326752817
-2 0.1353353508704025
-1 0.3678796250961014
1 2.7182831874306146
2 7.38905979424942
0 1.0000004999621837
3 20.08554696786291


()

<hr />

## Tangent Line Equation

### The Equation of Tangent Lines

In [7]:
fn f(x: f64) -> f64 {
    // Define the curve equation here, for example, a quadratic function
    // Let's use f(x) = x^2 - 3x + 2 as an example
    x.powi(2) - 3.0 * x + 2.0
}

In [8]:
fn slope(x1: f64, y1: f64, x2: f64, y2: f64) -> f64 {
    (y2 - y1) / (x2 - x1)
}

In [9]:
fn derivative(f: fn(f64) -> f64, x: f64) -> f64 {
    const EPSILON: f64 = 1e-8;
    let x1 = x - EPSILON;
    let x2 = x + EPSILON;
    let y1 = f(x1);
    let y2 = f(x2);
    (y2 - y1) / (x2 - x1)
}

In [10]:
fn tangent_line(f: fn(f64) -> f64, x: f64) {
    let m = derivative(f, x);
    let y0 = f(x);
    let b = y0 - m * x;

    println!("Equation of the tangent line at x = {:.2} is: y = {:.2}x + {:.2}", x, m, b);
}

In [11]:
let point_a_x = -0.48;
let point_b_x = 0.67;

tangent_line(f, point_a_x);
tangent_line(f, point_b_x);

Equation of the tangent line at x = -0.48 is: y = -3.96x + 1.77
Equation of the tangent line at x = 0.67 is: y = -1.66x + 1.55


<hr />

## Calculating Integrals

### Using Trapezoids

1. Convert angle from degrees to radians:
 
#### $\text{angle_in_radians} = \text{angle} \times \frac{\pi}{180}$

#### $\text{arc_length} = \text{radius} \times \text{angle_in_radians}$

3. Calculate the base length:

#### $\text{base_length} = 2 \times \text{radius} \times \sin(\text{angle_in_radians})$

4. Numerical integration using trapezoidal rule:

#### $\text{integral} = \text{trap_integral}(f, 0, \text{angle_in_radians}, 100)$

   Where 

#### $ f(x) = \text{radius} \times \cos(\text{angle_in_radians}) \times x$

5. Area of circle sector:

#### $\text{area} = \text{integral} - 0.5 \times \text{base_length} \times \text{arc_length}$

### Exercise 1: Finding the Area of a Circle Sector Using Trapezoids

In [17]:
use std::f64::consts::PI;

fn circle_sector_area(radius: f64, angle: f64) -> f64 {
    let angle_in_radians = angle * PI / 180.0;
    let arc_length = radius * angle_in_radians;
    let base_length = 2.0 * radius * angle_in_radians.sin();

    trap_integral(|x| radius * angle_in_radians.cos() * x, 0.0, angle_in_radians, 100) - 0.5 * base_length * arc_length
}

fn trap_integral<F>(f: F, a: f64, b: f64, num_trapezoids: usize) -> f64
where
    F: Fn(f64) -> f64,
{
    let width = (b - a) / num_trapezoids as f64;
    let mut integral = 0.0;

    for i in 0..num_trapezoids {
        let x0 = a + i as f64 * width;
        let x1 = x0 + width;
        let y0 = f(x0);
        let y1 = f(x1);
        integral += 0.5 * width * (y0 + y1);
    }

    integral
}

    let radius = 5.0;
    let angles = [30.0, 45.0, 60.0, 90.0, 120.0, 150.0];

    for angle in angles.iter() {
        let area = circle_sector_area(radius, *angle).abs();
        println!("Area of a {}° sector: {:.2}", angle, area);
    }


Area of a 30° sector: 5.95
Area of a 45° sector: 12.79
Area of a 60° sector: 21.30
Area of a 90° sector: 39.27
Area of a 150° sector: 47.56
Area of a 120° sector: 50.83


()

### Exercise 2: Finding the Area of a Circle Sector Using Simpson's Rule

1. Convert angle from degrees to radians:

#### $\text{angle_in_radians} = \text{angle} \times \frac{\pi}{180}$

2. Calculate the arc length:

#### $\text{arc_length} = \text{radius} \times \text{angle_in_radians}$

3. Calculate the base length:

#### $\text{base_length} = 2 \times \text{radius} \times \sin(\text{angle_in_radians})$

4. Numerical integration using Simpson's rule:

#### $\text{integral} = \text{simpson_integral}(f, 0, \text{angle_in_radians}, 100)$

Where

#### $f(x) = \text{radius} \times \cos(\text{angle_in_radians}) \times x$

5. Area of circle sector:

#### $\text{area} = \text{integral} - 0.5 \times \text{base_length} \times \text{arc_length}$

In [19]:
use std::f64::consts::PI;

fn circle_sector_area(radius: f64, angle: f64) -> f64 {
    let angle_in_radians = angle * PI / 180.0;
    let arc_length = radius * angle_in_radians;
    let base_length = 2.0 * radius * angle_in_radians.sin();

    simpson_integral(|x| radius * angle_in_radians.cos() * x, 0.0, angle_in_radians, 100) - 0.5 * base_length * arc_length
}

fn simpson_integral<F>(f: F, a: f64, b: f64, num_intervals: usize) -> f64
where
    F: Fn(f64) -> f64,
{
    let h = (b - a) / (2.0 * num_intervals as f64);
    let mut integral = f(a) + f(b);

    for i in 1..(2 * num_intervals) {
        let x = a + h * i as f64;
        integral += if i % 2 == 0 { 2.0 } else { 4.0 } * f(x);
    }

    integral *= h / 3.0;
    integral
}

let radius = 5.0;
let angles = [30.0, 45.0, 60.0, 90.0, 120.0, 150.0];

for angle in angles.iter() {
    let area = circle_sector_area(radius, *angle).abs();
    println!("Area of a {}° sector: {:.2}", angle, area);
}

Area of a 30° sector: 5.95
Area of a 45° sector: 12.79
Area of a 60° sector: 21.30
Area of a 90° sector: 39.27
Area of a 120° sector: 50.83
Area of a 150° sector: 47.56


()

### Exercise 3: Finding the Area of a Circle Sector Using A Direct Formula

#### $\text{Area} = \frac{\text{angle_in_radians}}{2} \times \text{radius}^2$

In [20]:
use std::f64::consts::PI;

fn circle_sector_area(radius: f64, angle: f64) -> f64 {
    let angle_in_radians = angle * PI / 180.0;
    let area = (angle_in_radians / 2.0) * radius.powf(2.0);
    area
}

    let radius = 5.0;
    let angles = [30.0, 45.0, 60.0, 90.0, 120.0, 150.0];

for angle in angles.iter() {
    let area = circle_sector_area(radius, *angle).abs();
    println!("Area of a {}° sector: {:.2}", angle, area);
}

Area of a 30° sector: 6.54
Area of a 45° sector: 9.82
Area of a 60° sector: 13.09
Area of a 90° sector: 19.63
Area of a 150° sector: 32.72
Area of a 120° sector: 26.18


()

<hr />

### Using Integrals to Solve Applied Problems

In [21]:
fn f(x: f64) -> f64 {
    x.sqrt()
}

In [22]:
use std::f64::consts::PI;

fn vol_solid(f: fn(f64) -> f64, a: f64, b: f64) -> f64 {
    let mut volume = 0.0;
    let num_slices = 1000;
    let width = (b - a) / num_slices as f64;
    for i in 0..num_slices {
        let x = a + i as f64 * width;
        let r = f(x);
        let vol = PI * r.powi(2) * width;
        volume += vol;
    }
    volume
}

In [23]:
let result = vol_solid(f, 0.0, 1.0);
result

1.5692255304681022

### Exercise: Volume of a Cone

1. Line segment `f(x) = x`:

#### $f(x) = x$

2. Volume of a cone using a given number of cylinders:

#### $\text{vol_cone}(f, a, b) = \sum_{i=0}^{n-1} \pi \cdot r_i^2 \cdot \text{width}$

Where:
- `f` represents the function for the line segment.
- `a` and `b` are the interval limits for the line segment.
- `n` is the number of cylinders (num\_cylinders in Rust code).
- `r_i` is the value of `f(x)` for each cylinder.
- `width` is the width of each cylinder, calculated as `(b - a) / n`.

In [24]:
fn f(x: f64) -> f64 {
    x
}

In [25]:
fn vol_cone(f: fn(f64) -> f64, a: f64, b: f64) -> f64 {
    let mut volume = 0.0;
    let num_cylinders = 10000;
    let width = (b - a) / num_cylinders as f64;

    for i in 0..num_cylinders {
        let x = a + i as f64 * width;
        let r = f(x);
        let vol = PI * r.powi(2) * width;
        volume += vol;
    }

    volume
}

In [26]:
let result = vol_cone(f, 0.0, 5.0);
println!("Volume of the cone: {:.2}", result);

Volume of the cone: 130.88


<hr />

## Using Derivatives to Solve Optimization Problems

### Exercise 1: Maximizing a Cubic Function

The Newton-Raphson method is based on the following iteration formula:

#### $x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}$

Where:
- `x_n` is the current approximation for the critical point (initially set to the `initial_guess`).
- `x_{n+1}` is the next approximation.
- `f(x_n)` and `f'(x_n)` are the function value and derivative at `x_n`, respectively.


1. Cubic function:

#### $f(x) = -x^3 + 3x^2 + 9x - 8$

2. Derivative of the cubic function:

#### $f'(x) = -3x^2 + 6x + 9$

3. Newton-Raphson iteration formula:

#### $x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}$

In [27]:
fn f(x: f64) -> f64 {
    -x.powi(3) + 3.0 * x.powi(2) + 9.0 * x - 8.0
}

fn f_prime(x: f64) -> f64 {
    -3.0 * x.powi(2) + 6.0 * x + 9.0
}

fn newton_method(f: fn(f64) -> f64, f_prime: fn(f64) -> f64, initial_guess: f64, max_iterations: usize) -> f64 {
    let mut x = initial_guess;
    let mut iterations = 0;
    let tolerance = 1e-6;

    while iterations < max_iterations {
        let delta = f(x) / f_prime(x);
        x -= delta;

        if delta.abs() < tolerance {
            break;
        }

        iterations += 1;
    }

    x
}

let critical_point = newton_method(f, f_prime, 0.0, 100);
println!("Local maximum at x = {:.2}", critical_point);

Local maximum at x = 0.75


### Exercise 2: Minimizing Total Cost

1. Cost function:

#### $\text{cost}(x) = 5000 + 20x$

2. Revenue function:

#### $\text{revenue}(x) = 50x$

3. Profit function:

#### $\text{profit}(x) = \text{revenue}(x) - \text{cost}(x)$

4. Derivative of the profit function:

#### $\text{profit_prime}(x) = 50 - 20$

5. Newton-Raphson iteration formula:

#### $x_{n+1} = x_n - \frac{\text{profit}(x_n)}{\text{profit_prime}(x_n)}$

In [28]:
fn cost(x: f64) -> f64 {
    5000.0 + 20.0 * x
}

fn revenue(x: f64) -> f64 {
    50.0 * x
}

fn profit(x: f64) -> f64 {
    revenue(x) - cost(x)
}

fn profit_prime(x: f64) -> f64 {
    50.0 - 20.0
}

fn newton_method(f: fn(f64) -> f64, f_prime: fn(f64) -> f64, initial_guess: f64, max_iterations: usize) -> f64 {
    let mut x = initial_guess;
    let mut iterations = 0;
    let tolerance = 1e-6;

    while iterations < max_iterations {
        let delta = f(x) / f_prime(x);
        x -= delta;

        if delta.abs() < tolerance {
            break;
        }

        iterations += 1;
    }

    x
}

let optimal_price = newton_method(profit, profit_prime, 50.0, 100);
println!("Optimal price: ${:.2}", optimal_price);

Optimal price: $166.67


### Exercise 3: Maximizing Profit in a Market

1. Demand Function:

The demand function `demand(x)` represents the quantity of the product demanded by consumers at a given price `x`. It is defined as:

#### $\text{demand}(x) = 100 - x$

2. Supply Function:

The supply function `supply(x)` represents the quantity of the product supplied by producers at a given price `x`. It is defined as:

#### $\text{supply}(x) = 20 + 2x$

3. Profit Function:|

The profit function `profit(x)` calculates the profit obtained from selling `x` units of the product at a given price. It is defined as the difference between the revenue (price times quantity) and the total cost:

#### $\text{profit}(x) = \text{demand}(x) \cdot x - (20 + 2x) \cdot x$

4. Derivative of the Profit Function:

To maximize the profit, we find the critical points of the profit function. The derivative of the profit function with respect to `x`, denoted as `profit_prime(x)`, represents the rate of change of profit with respect to the quantity `x`:

#### $\text{profit_prime}(x) = \text{demand}(x) - 2x - (20 + 2x)$

In [29]:
fn demand(x: f64) -> f64 {
    100.0 - x
}

fn supply(x: f64) -> f64 {
    20.0 + 2.0 * x
}

fn price(x: f64) -> f64 {
    (demand(x) + supply(x)) / 2.0
}

fn profit(x: f64) -> f64 {
    price(x) * x - (20.0 + 2.0 * x) * x
}

fn profit_prime(x: f64) -> f64 {
    100.0 - 4.0 * x
}

fn newton_method(f: fn(f64) -> f64, f_prime: fn(f64) -> f64, initial_guess: f64, max_iterations: usize) -> f64 {
    let mut x = initial_guess;
    let mut iterations = 0;
    let tolerance = 1e-6;

    while iterations < max_iterations {
        let delta = f(x) / f_prime(x);
        x -= delta;

        if delta.abs() < tolerance {
            break;
        }

        iterations += 1;
    }

    x
}

let equilibrium_quantity = newton_method(profit, profit_prime, 20.0, 100);
println!("Equilibrium quantity: {:.2} units", equilibrium_quantity);

Equilibrium quantity: 0.00 units


### Exercise 4: Minimizing Loss

1. Stock Value Function:

The stock value function `stock_value(t)` represents the value of the stock at time `t`. It is defined as:

#### $\text{stock_value}(t) = 1000 - 20t + 3t^2$

2. Stock Loss Function:

The stock loss function `stock_loss(t)` represents the loss incurred from holding the stock at time `t`. It is defined as the difference between the initial investment (1000 units) and the value of the stock at time `t`:

#### $\text{stock_loss}(t) = 1000 - \text{stock_value}(t)$

3. Derivative of the Stock Value Function:

To find the time of minimum loss, we need to find the critical point of the stock loss function. For that, we calculate the derivative of the stock value function with respect to `t`, denoted as `stock_value_prime(t)`. It represents the rate of change of the stock value with respect to time `t`:

#### $\text{stock_value_prime}(t) = -20 + 6t$

4. Derivative of the Stock Loss Function:

The derivative of the stock loss function with respect to `t`, denoted as `stock_loss_prime(t)`, can be obtained as the negative of the derivative of the stock value function:

#### $\text{stock_loss_prime}(t) = -\text{stock_value_prime}(t)$

5. Newton-Raphson Method:

The Newton-Raphson method is used to find the time of minimum loss, which corresponds to the critical point of the stock loss function.

6. Finding Time of Minimum Loss and Minimum Loss Amount:

In the `main` function, we use the `newton_method` to find the time at which the stock experiences the minimum loss and calculate the corresponding minimum loss amount using the `stock_loss` function:

#### $\text{Time of minimum loss:} \text{time_min_loss} \text{months}$
#### $\text{Minimum loss:} \text{{min_loss}}$

In [30]:
fn stock_value(t: f64) -> f64 {
    1000.0 - 20.0 * t + 3.0 * t.powi(2)
}

fn stock_loss(t: f64) -> f64 {
    1000.0 - stock_value(t)
}

fn stock_value_prime(t: f64) -> f64 {
    -20.0 + 6.0 * t
}

fn stock_loss_prime(t: f64) -> f64 {
    -stock_value_prime(t)
}

fn newton_method(f: fn(f64) -> f64, f_prime: fn(f64) -> f64, initial_guess: f64, max_iterations: usize) -> f64 {
    let mut x = initial_guess;
    let mut iterations = 0;
    let tolerance = 1e-6;

    while iterations < max_iterations {
        let delta = f(x) / f_prime(x);
        x -= delta;

        if delta.abs() < tolerance {
            break;
        }

        iterations += 1;
    }

    x
}

let time_min_loss = newton_method(stock_loss, stock_loss_prime, 5.0, 100);
let min_loss = stock_loss(time_min_loss);

println!("Time of minimum loss: {:.2} months", time_min_loss);
println!("Minimum loss: ${:.2}", min_loss);

Time of minimum loss: 6.67 months
Minimum loss: $-0.00


### Exercise 5: Maximizing Crop Yield

1. Objective Function:

#### $\text{crop_yield}(length, width) = \text{length} \times \text{width}$

2. Partial Derivatives:

#### $\frac{\partial \text{crop_yield}}{\partial \text{length}} = \text{width}$
#### $\frac{\partial \text{crop_yield}}{\partial \text{width}} = \text{length}$

3. Jacobian Determinant:
#### $\text{determinant} = \frac{\partial \text{crop_yield}}{\partial \text{length}} \times \frac{\partial \text{crop_yield}}{\partial \text{width}} - \frac{\partial \text{crop_yield}}{\partial \text{width}} \times \frac{\partial \text{crop_yield}}{\partial \text{length}}$

4. Newton's Method Updates:
#### $\text{length}_{\text{new}} = \text{length}_{\text{old}} - \frac{\frac{\partial \text{crop_yield}}{\partial \text{width}} \times \Delta y - \frac{\partial \text{crop_yield}}{\partial \text{length}} \times \Delta x}{\text{determinant}}$
#### $\text{width}_{\text{new}} = \text{width}_{\text{old}} - \frac{\frac{\partial \text{crop_yield}}{\partial \text{length}} \times \Delta x - \frac{\partial \text{crop|_yield}}{\partial \text{width}} \times \Delta y}{\text{determinant}}$

In [31]:
fn crop_yield(length: f64, width: f64) -> f64 {
    length * width
}

fn crop_yield_prime_length(length: f64, width: f64) -> f64 {
    width
}

fn crop_yield_prime_width(length: f64, width: f64) -> f64 {
    length
}

fn newton_method_2d(
    f_prime_x: fn(f64, f64) -> f64,
    f_prime_y: fn(f64, f64) -> f64,
    initial_guess_x: f64,
    initial_guess_y: f64,
    max_iterations: usize,
) -> (f64, f64) {
    let mut x = initial_guess_x;
    let mut y = initial_guess_y;
    let tolerance = 1e-6;

    for _ in 0..max_iterations {
        let delta_x = f_prime_x(x, y);
        let delta_y = f_prime_y(x, y);

        let determinant = delta_x * delta_y - crop_yield_prime_width(x, y) * crop_yield_prime_length(x, y);
        if determinant.abs() < 1e-6 {
            break;
        }

        let delta_length = (crop_yield_prime_width(x, y) * delta_y - delta_x * crop_yield_prime_length(x, y)) / determinant;
        let delta_width = (delta_x * crop_yield_prime_width(x, y) - delta_y * crop_yield_prime_length(x, y)) / determinant;

        x -= delta_length;
        y -= delta_width;

        if delta_length.abs() < tolerance && delta_width.abs() < tolerance {
            break;
        }
    }

    (x, y)
}

let (length, width) = newton_method_2d(crop_yield_prime_length, crop_yield_prime_width, 20.0, 20.0, 100);
let yield_max = crop_yield(length, width);

println!("Maximized crop yield: {:.2} square meters", yield_max);
println!("Dimensions of the rectangular field: {:.2} meters x {:.2} meters", length, width);

Maximized crop yield: 400.00 square meters
Dimensions of the rectangular field: 20.00 meters x 20.00 meters


<hr />